In [ ]:
!pip install pyLDAvis --quiet
!pip install chart_studio --quiet


In [ ]:
import pandas as pd
import numpy as np
import time
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
import gensim
from spacy.tokenizer import Tokenizer
import gensim.corpora as corpora
from gensim.models.ldamulticore import LdaMulticore
from pprint import pprint
from gensim.models.coherencemodel import CoherenceModel
import plotly.express as px
import pyLDAvis.gensim
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls
from operator import itemgetter
from ipywidgets import interact
import tqdm
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets


# supress warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We will load in our preprocessed tweets from our [data cleaning notebook](https://github.com/tarrantcarter/Final_Capstone/blob/main/Modern_Motivation_Data_Cleaning_Feature_Engineering.ipynb). The csv can be found [here](https://drive.google.com/file/d/1-d-61YuocweY0F3rSnf-idL1ixbuKq4H/view?usp=sharing). 

In [ ]:
# load in cleaned tweets from data cleaning notebook
tweets_cleaned = pd.read_json("/content/drive/MyDrive/Data/NLP_Capstone/motivational_tweets_cleaned_bigram.json")

In [ ]:
tweets_cleaned.head()

,date,user_name,content,content_preprocessed,unigram_tokens,ngrams,ngram_tokens,nouns_only,nouns_verbs,bigrams,bigram_strings
44,2021-01-17 22:13:17,LewisHowes,Know this. Everything is happening for a reaso...,know happen reason favor betterment future pai...,"[know, happen, reason, favor, betterment, futu...",know happen_reason favor betterment future pai...,"[know, happen_reason, favor, betterment, futur...","[reason, favor, betterment, future, pain, feel...","[know, reason, favor, betterment, future, pain...",[happen_reason],happen_reason
61,2021-01-15 15:28:06,LewisHowes,Protect your inner peace at all costs. Create ...,protect inner peace cost create daily practice...,"[protect, inner, peace, cost, create, daily, p...",protect inner_peace cost create daily_practice...,"[protect, inner_peace, cost, create, daily_pra...","[peace, cost, practice, communicate, stress, d...","[peace, cost, practice, communicate, stress, d...","[inner_peace, daily_practice]",inner_peace daily_practice
161,2021-01-07 16:00:29,LewisHowes,Always remember to ask for exactly what you wa...,remember ask exactly want ask love good health...,"[remember, ask, exactly, want, ask, love, good...",remember ask exactly want ask love good health...,"[remember, ask, exactly, want, ask, love, good...","[health, abundance, peace, ask, wisdom, creati...","[remember, health, abundance, peace, ask, wisd...","[health_abundance, ask_wisdom]",health_abundance ask_wisdom
274,2021-01-01 02:44:46,LewisHowes,This will be your greatest year ever. All your...,great year work start pay earn happy love deep...,"[great, year, work, start, pay, earn, happy, l...",great year work start pay earn happy love deep...,"[great, year, work, start, pay, earn, happy, l...","[year, work, start, earn, love, embrace, fear,...","[year, work, start, pay, earn, love, embrace, ...",[massive_action],massive_action
317,2020-12-30 01:38:35,LewisHowes,Be grateful for the breakdown this year. It's ...,grateful breakdown year set massive breakthrou...,"[grateful, breakdown, year, set, massive, brea...",grateful breakdown year set massive breakthrou...,"[grateful, breakdown, year, set, massive, brea...","[year, breakthrough, money, mission, get, rela...","[year, set, breakthrough, money, come, mission...",[real_friend],real_friend


In [ ]:
tweets_cleaned.applymap(type)

,date,user_name,content,content_preprocessed,unigram_tokens,ngrams,ngram_tokens,nouns_only,nouns_verbs,bigrams,bigram_strings
44,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
61,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
161,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
274,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
317,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...
977598,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977599,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977606,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977608,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>


In [ ]:
tweets_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 583340 entries, 44 to 977610
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   date                  583340 non-null  datetime64[ns]
 1   user_name             583340 non-null  object        
 2   content               583340 non-null  object        
 3   content_preprocessed  583340 non-null  object        
 4   unigram_tokens        583340 non-null  object        
 5   ngrams                583340 non-null  object        
 6   ngram_tokens          583340 non-null  object        
 7   nouns_only            583340 non-null  object        
 8   nouns_verbs           583340 non-null  object        
 9   bigrams               583340 non-null  object        
 10  bigram_strings        583340 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 53.4+ MB


In [ ]:
tweets_cleaned.shape

(583340, 11)

# Topic Modeling

In [ ]:
# create dictionary
id2word = corpora.Dictionary(tweets_cleaned['bigrams'])
# create texts corpus
texts = tweets_cleaned['bigrams']
# term document frequency
corpus = [id2word.doc2bow(text) for text in texts]
# print first 30 tuples from corpus
print(corpus[:1][0][:30])

[(0, 1)]


In [ ]:
# number of topics
num_topics = 10
# build LDA model
base_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

In [ ]:
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
away_self_respect_gandhi experience_have_send_benefit high_form_ignorance_reject_know open_guidance_true_nature_free abundance_acquire_tune_dyer amp_uncomplicated_dyer practice_keep release_feelings amp_guilt_dyer far_able_farther_morgan

------ Topic 1 ------
give_expectation refuse_let close_heart trust_heart follow_heart_quiet_ask_question_feel_answer_learn secret_success stumble_block_pessimist_step_stone_optimist_eleanor_roosevelt ready_opportunity come_benjamin_disraeli true_wisdom

------ Topic 2 ------
heaven_earth_choice_place detach_stuff_dyer thomas_edison irritate_lead_understanding_carl_jung happiness_exist_acceptance_denis_de_rougamont shoe_fit_person_pinch_recipe_live_suit_case_carl_jung discover_new_ocean_courage lose_sight_shore_gide better_better_maya_anjelou right_stick_george_eliot

------ Topic 3 ------
feeling_discouragement_prey_end_sure_succeed_abraham_lincoln ralph_waldo_emerson vincent_van_gogh love_cornerstone_having_abundance treat_karm

In [ ]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=tweets_cleaned['bigrams'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -8.883947435932908

Coherence Score:  0.7611483075609546


In [ ]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(base_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.148499  0.318087       1        1  22.058203
2      0.112361 -0.289129       2        1  12.861362
3      0.102716 -0.087394       3        1  10.890319
0      0.128076  0.182261       4        1   8.442245
5      0.085896 -0.090108       5        1   8.144758
6     -0.285035  0.024758       6        1   8.127262
1     -0.262519  0.023115       7        1   8.084841
8      0.060792 -0.045445       8        1   7.235452
7      0.071917 -0.048453       9        1   7.098335
4     -0.162703  0.012308      10        1   7.057224, topic_info=                               Term         Freq  ...  logprob  loglift
106                 albert_einstein  5763.000000  ...  30.0000  30.0000
4051  longer_dwell_misfortune_great  2986.000000  ...  29.0000  29.0000
753                  passion_trumps  2645.000000  ...  28.0000  28.0000
752                    excuses_dyer  2645.000000  ...  27.0000  27.0000
836      suit_wear_need_pocket_dyer  2434.000000  ...  26.0000  26.0000
...                             ...          ...  ...      ...      ...
746                  die_music_find   814.019231  ...  -4.2464   2.6497
839           capacity_attract_miss   810.523506  ...  -4.2507   2.6497
1925                  know_socrates   813.937679  ...  -4.2465   2.6491
3037              action_dalai_lama   814.689865  ...  -4.2456   2.6488
4358                happiness_ready   814.680651  ...  -4.2456   2.6488

[379 rows x 6 columns], token_table=      Topic      Freq                         Term
term                                              
25        9  0.999526              abraham_lincoln
954       4  0.999705  abundance_acquire_tune_dyer
779       4  0.998995   act_glorious_sensation_joy
3037      2  0.001225            action_dalai_lama
3037     10  0.998046            action_dalai_lama
...     ...       ...                          ...
904       1  0.000459  wisdom_avoid_thought_weaken
904       6  0.998946  wisdom_avoid_thought_weaken
3593     10  0.998114           wish_john_chambers
2791      3  0.999289               word_sum_learn
1354      5  0.999114                  world_large

[400 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 3, 4, 1, 6, 7, 2, 9, 8, 5])

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           alpha=a,
                                           eta=b,
                                           random_state=222)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
# start preprocess runtime
start_time = time.time() 

# Topics range
min_topics = 3
max_topics = 10
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = [.01,.1,.5,1,'symmetric','asymmetric']
# Beta parameter
beta = [.01,.1,.5,1,'symmetric']

# model results dict
model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=210)

    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        for a in alpha:
            # iterare through beta values
            for b in beta:
                # get the coherence score for the given parameters
                cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                              k=k, a=a, b=b)
                # Save the model results
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
                
                pbar.update(1)
    lda_tuning_results = pd.DataFrame(model_results)                
    lda_tuning_results.to_csv('/content/drive/MyDrive/Data/NLP_Capstone/lda_tuning_results3.csv', index=False)
    pbar.close()

# print preprocess runtime
print(time.strftime(f'%H hours, %M minutes, %S seconds', time.gmtime(time.time() - start_time)))


100%|██████████| 210/210 [2:18:14<00:00, 39.50s/it]

02 hours, 18 minutes, 14 seconds


In [ ]:
lda_tuning_results.sort_values('Coherence',ascending=False).head(30)

,Topics,Alpha,Beta,Coherence
5,3,0.1,0.01,0.840322
0,3,0.01,0.01,0.840315
20,3,symmetric,0.01,0.836824
35,4,0.1,0.01,0.832715
50,4,symmetric,0.01,0.830364
15,3,1,0.01,0.828682
10,3,0.5,0.01,0.828357
95,6,0.1,0.01,0.828270
110,6,symmetric,0.01,0.827735
30,4,0.01,0.01,0.827007


In [ ]:
# number optimal of topics
num_topics = 3
# build optimal LDA model
optimal_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                        alpha=.1,
                                        eta=.01,
                                        random_state=222
                                        )
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in optimal_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
heaven_earth_choice_place wayne_dyer peter_drucker winston_churchill experience_have_send_benefit amp_guilt_dyer give_expectation appreciate_beauty_touch_amp treat_karma_react_dyer amp_attention_visualize_door_begin_open

------ Topic 1 ------
suit_wear_need_pocket_dyer passion_trumps excuses_dyer peaceful_life detach_stuff_dyer henry_david_thoreau thomas_edison show_fulfill_destiny_act high_form_ignorance_reject_know bring_peace

------ Topic 2 ------
albert_einstein ralph_waldo_emerson abraham_lincoln wisdom_avoid_thought_weaken day_harvest_reap_seed_plant_robert_louis_stevenson far_able_farther_morgan tao_teach_storm_last_forever_matter love_cornerstone_having_abundance real_magic_relationships mean_absence_judgment



In [ ]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = optimal_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=optimal_model, texts=tweets_cleaned['bigrams'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -93.13177487820442

Coherence Score:  0.8403287165606262


In [ ]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.032674 -0.397155       1        1  34.809313
2     -0.361406  0.170373       2        1  33.003324
0      0.328732  0.226782       3        1  32.187364, topic_info=                          Term          Freq  ...  logprob  loglift
106            albert_einstein  23065.000000  ...  30.0000  30.0000
842  heaven_earth_choice_place   6529.000000  ...  29.0000  29.0000
644                 wayne_dyer   6397.000000  ...  28.0000  28.0000
449              peter_drucker   5738.000000  ...  27.0000  27.0000
88         ralph_waldo_emerson   5684.000000  ...  26.0000  26.0000
..                         ...           ...  ...      ...      ...
746             die_music_find   2185.962870  ...  -4.7761   1.1336
685              william_james   2196.244438  ...  -4.7714   1.1335
879             bother_problem   2185.151581  ...  -4.7764   1.1336
848        life_rehearsal_live   2185.991129  ...  -4.7761   1.1335
878        come_squeeze_inside   2185.203992  ...  -4.7764   1.1336

[138 rows x 6 columns], token_table=      Topic      Freq                                     Term
term                                                          
25        2  1.000052                          abraham_lincoln
954       1  0.999869              abundance_acquire_tune_dyer
106       2  0.999892                          albert_einstein
106       3  0.000087                          albert_einstein
800       3  1.000014  amp_attention_visualize_door_begin_open
...     ...       ...                                      ...
644       3  1.000056                               wayne_dyer
685       3  0.999812                            william_james
171       3  1.000031                        winston_churchill
904       2  1.000013              wisdom_avoid_thought_weaken
1354      3  0.999964                              world_large

[112 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])